# Run MatChain by API

## Read the datasets

This notebook illlustrates how to match two datasets A and B with MatChain's API and gives some information about the used algorithms and its parameters.

Firstly, change to MatChain's main directory and import some libraries.

In [1]:
%cd ..


d:\repos\matchain


In [2]:
import pandas as pd
import matchain.api


The datasets A and B used in this notebook contain bibliographic information about scientific articles. A and B are given in the form of CSV files and are read into Pandas' dataframes. The column 'id' refers to the row number but is not required and can be omitted.

In [3]:
data_dir = './data/Structured/DBLP-ACM'
dfa = pd.read_csv(f'{data_dir}/tableA.csv', index_col='id')
dfb = pd.read_csv(f'{data_dir}/tableB.csv', index_col='id')


In [4]:
dfa.head(5)


,title,authors,venue,year
id,,,,
0,semantic integration of environmental models f...,d. scott mackay,sigmod record,1999
1,estimation of query-result distribution and it...,"viswanath poosala , yannis e. ioannidis",vldb,1996
2,incremental maintenance for non-distributive a...,"themistoklis palpanas , richard sidle , hamid ...",vldb,2002
3,cost-based selection of path expression proces...,"zhao-hui tang , georges gardarin , jean-robert...",vldb,1996
4,benchmarking spatial join operations with spat...,"erik g. hoel , hanan samet",vldb,1995


In [5]:
dfb.head(5)


,title,authors,venue,year
id,,,,
0,the wasa2 object-oriented workflow management ...,"gottfried vossen , mathias weske",international conference on management of data,1999
1,a user-centered interface for querying distrib...,"isabel f. cruz , kimberly m. james",international conference on management of data,1999
2,"world wide database-integrating the web , corb...","athman bouguettaya , boualem benatallah , lily...",international conference on management of data,1999
3,xml-based information mediation with mix,"chaitan baru , amarnath gupta , bertram lud &#...",international conference on management of data,1999
4,the ccube constraint object-oriented database ...,"alexander brodsky , victor e. segal , jia chen...",international conference on management of data,1999


## Declare the matching properties and similarity functions

Initialize the MatChain object with the dataframes.

In [6]:
mat = matchain.api.MatChain(dfa, dfb)


2023-11-02 21:23:39,670 WARNING  logging to console only
2023-11-02 21:23:39,686 INFO     selected datasets=['test']
2023-11-02 21:23:39,701 INFO     running command=prepare
2023-11-02 21:23:39,703 INFO     setting seed=1
2023-11-02 21:23:39,706 INFO     cuda available=True, embedding_device=cuda
2023-11-02 21:23:39,708 INFO     size_1=2616, size_2=2294, concat df_data=4910
2023-11-02 21:23:39,709 INFO     finished command=prepare, time=0.007691383361816406


We want to match the datasets on the properties (columns) having the same name "year", "title", "authors", and "venue", respectively. 
To do so, we have to specify the matching columns and their similarity functions. In this example, we use ```equal``` for the integer-valued column "year" and ```shingle_tfidf``` for each of the remaining string-valued columns.

In [7]:
mat.property('year', simfct='equal')
mat.property('title', simfct='shingle_tfidf')
mat.property('authors', simfct='shingle_tfidf')
mat.property('venue', simfct='shingle_tfidf')


These similarity functions calculate scores between 0 and 1 for pairs of property values. The higher the score, the more similar two values are.  

Currently, the following similarity functions are implemented:
- **any type**
    - ```equal```: 1 if two values are equal and 0 otherwise.
- **numeric type** 
    - ```absolute```
    - ```relative```
- **string type**
    - ```fuzzy```: Uses library [thefuzz](https://github.com/seatgeek/thefuzz) (formerly known as fuzzywuzzy) to derive similarity values between two strings.
    - ```tfidf```: Segments each string into words and represents it with a sparse vector of TFIDF weights for its constituent words.
    - ```shingle_tfidf```: For each string, it computes shingles and a sparse vector with TFIDF weights for these shingles. Shingles are character-level n-grams. For example, the shingles for the string 'matchain' and $n=3$ are 'mat', 'atc', 'tch', 'cha', 'hai' and 'ain'. 
    - ```embedding```: For each string, [SentenceTransformer](https://github.com/UKPLab/sentence-transformers) generates a dense embedding vector.


In case of ```tfidf```, ```shingle_tfidf```, and ```embedding```, similarity scores are determined as cosine similarities between the vector representations of two strings.

If the ```property``` method is called several times for the same property name, the similarity scores are aggregated for this property. 

## Blocking

As the total number of record pairs grows with the product of the record sizes in datasets A and B, classifying each pair as matching or non-matching can be computationally expensive, especially for large datasets. Blocking effectively reduces the number of pairs while only discarding a small fraction of true matching pairs. The reduced set of pairs is called candidate pairs.

In our example, we specify the three properties ```title```, ```authors```, and ```venue``` for blocking. The ```blocking``` method returns the candidate pairs as Pandas MultiIndex. The MultiIndex is sorted in ascending order by the first index which refers to the row index of the first dataset. The second index refers to the row index of the second dataset shifted by the number of rows of the first dataset.

In [8]:
candidate_pairs = mat.blocking(blocking_props=['title', 'authors', 'venue'])
candidate_pairs


2023-11-02 21:23:42,106 INFO     running command=blocking
2023-11-02 21:23:42,303 INFO     generated vectors=(6567, 8257), time=0.17798328399658203, df_values=(4910, 3), df_index_array=(4910, 3), values=6567
2023-11-02 21:23:42,367 INFO     blocking prop=title, new candidates=3649, all candidates=3649, total time nn search=0.06341695785522461
2023-11-02 21:23:42,419 INFO     blocking prop=authors, new candidates=8067, all candidates=9230, total time nn search=0.11521339416503906
2023-11-02 21:23:42,439 INFO     blocking prop=venue, new candidates=5200, all candidates=14385, total time nn search=0.13530826568603516
2023-11-02 21:23:42,452 INFO     candidate pairs=14385
2023-11-02 21:23:42,453 INFO     finished command=blocking, time=0.34694766998291016


MultiIndex([(   0, 2733),
            (   0, 4522),
            (   0, 4525),
            (   1, 3415),
            (   1, 3449),
            (   1, 3641),
            (   1, 3709),
            (   1, 4130),
            (   1, 4595),
            (   2, 3052),
            ...
            (2614, 4870),
            (2614, 4872),
            (2614, 4874),
            (2614, 4876),
            (2614, 4894),
            (2614, 4896),
            (2614, 4898),
            (2614, 4900),
            (2614, 4908),
            (2615, 4022)],
           names=['idx_1', 'idx_2'], length=14385)

In our example, blocking reduces the number of all pairs from around 6 million to the feasible size of 14.385 candidate pairs.

In [9]:
print('\nall pairs =', len(dfa) * len(dfb))
print('candidate pairs =', len(candidate_pairs))



all pairs = 6001104
candidate pairs = 14385


Currently, the ```blocking``` method allows to configure the following blocking algorithms: 

- **Token-Blocking**: 
    - Segments the strings for each blocking property into words (tokens) as done for similarity function ```tfidf```. Two records are considered as candidate pairs if they share at least one token.
    - ```name='token'```
    - All other parameters are ignored.
- **NN on sparse vectors**: 
    - NN stands for nearest neighbour search: For each vector with respect to a string from one dataset, find the nearest vectors in relation to strings from the other dataset. Finally, all found pairs having a similarity score (cosine similarity) above a specified threshold are returned as candidate pairs.
    - ```vector_type='shingle_tfidf'```, i.e. strings are represented as shingle vectors as described previously for similarity function ```shingle_tfidf```.
    - ```name='sparsedottopn'``` (for library [sparse_dot_topn](https://github.com/ing-bank/sparse_dot_topn)) or ```name='nmslib'``` (for library [NMSLIB](https://github.com/nmslib/nmslib)) or ```name='sklearn'``` (for [brute force NN with sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html))
    - Parameter ```shingle_size``` is the number of characters per shingle.
- **NN on dense vectors**: 
    - ```vector_type='embedding'```, i.e. strings are represented as embedding vectors as described previously for similarity function ```embedding```.
    - ```name='faiss'``` (for library [Faiss](https://github.com/facebookresearch/faiss)) or ```name='sklearn'``` (for [brute force NN with sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html))
    - Parameters ```embedding_batch_size```, ```embedding_model```, and ```embedding_device``` are delegated to [SentenceTransformer](https://github.com/UKPLab/sentence-transformers).

Brute force NN search finds the exact nearest neighbours but is too expensive for large datasets. By contrast, the other NN libraries usually perform only an approximate nearest neighbour search but are very fast.

The ```blocking``` method declares several optional parameters. By default, it uses the following parameter values:

```
    mat.blocking(name='sparsedottopn',
                vector_type='shingle_tfidf',
                shingle_size=3,
                query_strategy='smaller',
                ntop=10,
                blocking_threshold=0.5)
```

Here, query strategy ```smaller``` means that for each string from the smaller dataset, (approximate) nearest neighbours from the larger dataset are searched for. Alternatively, you may choose the query strategy ```larger```, ```first```, or ```second```.  The parameter ```ntop``` specifies the number of nearest neighbours to be found. The parameter ```blocking_threshold``` determines the threshold for the similarity score of a candidate pair, i.e. only NN pairs with a similarity score greater than the threshold are returned as candidate pairs.

## AutoCal

AutoCal stands for AutoCalibration and is an unsupervised matching algorithm that is based on statistics and heuristics. It is described [here](https://como.ceb.cam.ac.uk/preprints/293/) in detail. "Like many ML approaches, AutoCal expects the similarity features of properties as input. Since data sets frequently consist
of a mixture of expressive properties such as plant name or locality and more or less discriminative properties, AutoCal *calibrates* automatically the similarity feature values such that they become directly comparable and summable, and uses the resulting total scores for predicting matches."

MatChain provides a new and highly efficient implementation of AutoCal. When calling the ```autocal``` method, 
first the similarity feature values (i.e. the similarity scores) are computed for all candidate pairs and then AutoCal is started. Read the logging output of the next cell to check that both steps (commands) *similarity* and *autocal* were actually executed. However, you can also run the first step explicitly by calling the ```similarity``` method before the ```autocal``` method. 

In [10]:
#mat.similarity()
mat.autocal()


2023-11-02 21:23:45,944 INFO     running command=similarity
2023-11-02 21:23:45,947 INFO     computing vectorized similarity
2023-11-02 21:23:45,951 INFO     computed vectorized similarity=14385, sim columns=['0'], time=0.004377126693725586
2023-11-02 21:23:45,952 INFO     computing similarity tfidf
2023-11-02 21:23:46,256 INFO     computed similarity tfidf=14385, sim columns=['1', '2', '3'], time=0.3040890693664551
2023-11-02 21:23:46,263 INFO     finished command=similarity, time=0.319171667098999
2023-11-02 21:23:46,264 INFO     running command=autocal
2023-11-02 21:23:46,266 INFO     calculated maximum similarity, dataset_id=1, entities=2465
2023-11-02 21:23:46,316 INFO     calculated auto calibrated scores
2023-11-02 21:23:46,321 INFO     identified best total scores, dataset_id=1
2023-11-02 21:23:46,321 INFO     calculated maximum similarity, dataset_id=2, entities=2291
2023-11-02 21:23:46,369 INFO     calculated auto calibrated scores
2023-11-02 21:23:46,376 INFO     identified 

## Prediction and Evaluation

The ```predict``` method returns those candidate pairs which the matching algorithm classified as matching pairs. Once again, Pandas' MultiIndex is used to represent the matching pairs. However, this time both indices refer to the row number of the first and second dataset, respectively (without any offset).

In [11]:
predicted_matches = mat.predict()
print('\nnumber of predicted matches=', len(predicted_matches))
print('some predicted matches=', list(predicted_matches[:5]))



number of predicted matches= 2192
some predicted matches= [(0, 117), (1, 1093), (3, 1125), (4, 1450), (5, 49)]


In case of our example, the data directory ```data_dir``` also contains files for training, validation, and testing (which can be used for supervised learning). These are CSV files with pairs of row indices labeled as 0 (non-matching pair) and 1 (matching pair). The totality of all matching pairs can be used as ground truth for the unsupervised matching algorithm AutoCal. The ```evaluate``` method compares the predicted matches with the ground truth matches and computes the following metrics: F1-score *f1*, precision *p*, recall *r*, and the numbers of true positives *tpos*, false positives *fpos*, and false negatives *fneg*. Additionally, *t* stands for the matching threshold estimated by AutoCal. 

In [12]:
result = mat.evaluate(matches=data_dir)
print('\nmetrics=', result['union_set']['estimated'])


2023-11-02 21:23:49,592 INFO     running command=evaluate
2023-11-02 21:23:49,654 INFO     metrics=
{'blocking': {'matches': 2220, 'nonmatches': 10143, 'diff_matches': 4, 'diff_nonmatches': 8509, 'candidate_matches': 2216, 'candidate_nonmatches': 12169}, 'test_set': {'estimated': {'t': 0.425, 'f1': 0.97511, 'p': 0.97955, 'r': 0.97072, 'tpos': 431, 'fpos': 9, 'fneg': 13}}, 'union_set': {'estimated': {'t': 0.425, 'f1': 0.97416, 'p': 0.98038, 'r': 0.96802, 'tpos': 2149, 'fpos': 43, 'fneg': 71}}, 'match_frequencies_1_to_2': {0: 396, 1: 2220}, 'match_frequencies_2_to_1': {0: 74, 1: 2220}}
2023-11-02 21:23:49,654 INFO     finished command=evaluate, time=0.062146663665771484



metrics= {'t': 0.425, 'f1': 0.97416, 'p': 0.98038, 'r': 0.96802, 'tpos': 2149, 'fpos': 43, 'fneg': 71}


## PinBoard

Each command is executed as soon as the corresponding method such as ```blocking```, ```similarity```, or ```autocal``` is called. If a previous step is required but was not called yet, it is executed with default values automatically. Each step stores its intermediate results in a ```board``` object. This ```board``` object can be used to inspect the current state and to access or manipulate intermediate results.

For instance, the next line shows how to access the set of candidate pairs after the ```blocking``` method was called. 

For more details, see the documentation of class ```matchain.base.PinBoard```.

In [13]:
mat.board.candidate_pairs


MultiIndex([(   0, 2733),
            (   0, 4522),
            (   0, 4525),
            (   1, 3415),
            (   1, 3449),
            (   1, 3641),
            (   1, 3709),
            (   1, 4130),
            (   1, 4595),
            (   2, 3052),
            ...
            (2614, 4870),
            (2614, 4872),
            (2614, 4874),
            (2614, 4876),
            (2614, 4894),
            (2614, 4896),
            (2614, 4898),
            (2614, 4900),
            (2614, 4908),
            (2615, 4022)],
           names=['idx_1', 'idx_2'], length=14385)